# Importing the libraries:

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM,Dropout, Bidirectional
from tensorflow.keras.layers import Dense

# Importing the dataset:

In [2]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')


## Preprocessing the dataset:

In [3]:
# dropping the NANs values in dataset:
train_df = train_df.dropna()

# get the independent/dependent features from the dataset:
X = train_df.iloc[:,:-1]
y = train_df.iloc[:,-1]

In [4]:
X

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [5]:
y

0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 18285, dtype: int64

In [6]:
# Vocabulary_size / dictionary_size:
Vocabulary_size = 5000

# Prepare OneHotRepresentation:

## Prepare the Corpus:

In [7]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [8]:
messages = X.copy()
messages.reset_index(inplace=True)
messages

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...,...
18280,20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
18281,20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
18282,20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
18283,20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [9]:
ps = PorterStemmer()
corpus = []

for i in range(0, len(X)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [10]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

## Create oneHotRepresentation

In [11]:
oneHotRepres_messages = [one_hot(words,Vocabulary_size) for words in corpus]
oneHotRepres_messages

[[3150, 3266, 2606, 2303, 16, 4552, 440, 9, 2455, 3421],
 [1444, 841, 59, 4036, 2682, 2256, 1324],
 [4118, 4019, 1182, 493],
 [3314, 4048, 2528, 1835, 2921, 1007],
 [3343, 2682, 1754, 1211, 2135, 1454, 2682, 845, 1228, 4486],
 [1094,
  4732,
  2538,
  3609,
  1778,
  2408,
  4689,
  3560,
  2345,
  3389,
  4859,
  42,
  693,
  1320,
  1324],
 [4823, 1270, 3291, 4640, 2152, 2680, 3744, 1954, 3945, 4240, 1489],
 [3190, 1905, 34, 3600, 1932, 3655, 2408, 4553, 3945, 4240, 1489],
 [3459, 2463, 742, 950, 3, 592, 2015, 1713, 2408, 1913],
 [2456, 3386, 4977, 4639, 3975, 3729, 1856, 4191],
 [2579, 2722, 1462, 385, 1982, 4545, 3733, 1659, 4870, 4666, 2760],
 [1835, 583, 16, 592, 2408, 1932],
 [2077, 4386, 4148, 3922, 4644, 14, 4004, 488, 4255],
 [3807, 3577, 915, 2568, 1891, 796, 496, 3945, 4240, 1489],
 [3408, 4383, 99, 1325, 3341, 3945, 4240, 1489],
 [2282, 816, 2651, 4342, 128, 4861, 3377, 3179, 4960, 4402],
 [2333, 4087, 841],
 [1382, 384, 4690, 2323, 2408, 1532, 682, 1324],
 [1194, 1473, 59

# Prepare the padded_X from oneHotRepres_messages using pad_sequences:

In [12]:
fixed_title_sentence_len = 20

padded_X = pad_sequences(
    sequences=oneHotRepres_messages,
    padding='pre',
    maxlen=fixed_title_sentence_len,
)

In [13]:
padded_X

array([[   0,    0,    0, ...,    9, 2455, 3421],
       [   0,    0,    0, ..., 2682, 2256, 1324],
       [   0,    0,    0, ..., 4019, 1182,  493],
       ...,
       [   0,    0,    0, ..., 3945, 4240, 1489],
       [   0,    0,    0, ..., 2298, 4155, 3017],
       [   0,    0,    0, ...,  783,  457,  903]], dtype=int32)

# Prepare the Deeplearning Model:

In [14]:
total_features=40

model = Sequential()
model.add(Embedding(
    input_dim=Vocabulary_size,
    output_dim=total_features,
    input_length=fixed_title_sentence_len,
))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(100)))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])

print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
dropout (Dropout)            (None, 20, 40)            0         
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               112800    
_________________________________________________________________
dropout_1 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 201       
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [15]:
# create X_final and y_final:

X_final = np.array(padded_X)
y_final = np.array(y)

In [16]:
X_final.shape, y_final.shape

((18285, 20), (18285,))

# Splitting the Training/Testing dset split:

In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.3, random_state=42)

In [18]:
X_train, y_train

(array([[   0,    0,    0, ..., 1835, 2385, 1719],
        [   0,    0,    0, ..., 2408, 4863, 1324],
        [   0,    0,    0, ..., 3378,   81,  410],
        ...,
        [   0,    0,    0, ..., 3945, 4240, 1489],
        [   0,    0,    0, ..., 2138, 4533, 1324],
        [   0,    0,    0, ..., 1754, 2633, 1258]], dtype=int32),
 array([1, 0, 1, ..., 0, 0, 1]))

In [19]:
X_test, y_test

(array([[   0,    0,    0, ..., 1029, 3720, 3384],
        [   0,    0,    0, ..., 3945, 4240, 1489],
        [   0,    0,    0, ..., 3945, 4240, 1489],
        ...,
        [   0,    0,    0, ...,  282,   59, 2408],
        [   0,    0,    0, ..., 3380, 4195, 1324],
        [   0,    0,    0, ..., 2019, 3548, 2532]], dtype=int32),
 array([1, 0, 0, ..., 1, 0, 1]))

# Traning the created DL model:

In [20]:
model.fit(
    X_train, y_train,
    validation_data=(X_test,y_test),
    epochs=50,
    batch_size=32)

Epoch 1/50
400/400 [==============================] - 14s 12ms/step - loss: 0.3762 - accuracy: 0.8106 - val_loss: 0.2069 - val_accuracy: 0.9136
Epoch 2/50
400/400 [==============================] - 4s 10ms/step - loss: 0.1399 - accuracy: 0.9446 - val_loss: 0.1996 - val_accuracy: 0.9187
Epoch 3/50
400/400 [==============================] - 4s 10ms/step - loss: 0.0908 - accuracy: 0.9682 - val_loss: 0.2125 - val_accuracy: 0.9078
Epoch 4/50
400/400 [==============================] - 4s 10ms/step - loss: 0.0678 - accuracy: 0.9775 - val_loss: 0.2717 - val_accuracy: 0.9120
Epoch 5/50
400/400 [==============================] - 4s 11ms/step - loss: 0.0462 - accuracy: 0.9832 - val_loss: 0.2766 - val_accuracy: 0.9141
Epoch 6/50
400/400 [==============================] - 4s 10ms/step - loss: 0.0345 - accuracy: 0.9887 - val_loss: 0.2976 - val_accuracy: 0.9149
Epoch 7/50
400/400 [==============================] - 4s 10ms/step - loss: 0.0242 - accuracy: 0.9918 - val_loss: 0.3440 - val_accuracy: 0.914

# Evaluating the Performance on the test dataset

In [21]:
y_pred = model.predict_classes(X_test)
y_pred

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


array([[1],
       [0],
       [0],
       ...,
       [1],
       [0],
       [1]], dtype=int32)

In [22]:
from sklearn.metrics import confusion_matrix,accuracy_score

print(confusion_matrix(y_pred, y_test))

print("AccuracyScore on test Data: {:.2f}%".format(accuracy_score(y_pred,y_test)*100))


[[2836  182]
 [ 271 2197]]
AccuracyScore on test Data: 91.74%
